In [ ]:
import json
import os
import re

import dotenv
import matplotlib.pyplot as plt
import openai
import pandas as pd
import pyrootutils
import seaborn as sns

In [ ]:
PROJECT_ROOT = pyrootutils.find_root(
    search_from=os.path.abspath(""), indicator=".project-root"
)
GRAMMARS_PATH = PROJECT_ROOT / "data" / "scfg_grammars"

results_files = list(GRAMMARS_PATH.rglob("batch_*.jsonl"))
inputs_files = list(GRAMMARS_PATH.rglob("scfg_*.jsonl"))

dotenv.load_dotenv(PROJECT_ROOT / ".env")

In [ ]:
openai_client = openai.OpenAI()

In [ ]:
res_dfs = []
inputs_dfs = []

for f in results_files:
    df = pd.read_json(f, lines=True)
    json_struct = json.loads(df.to_json(orient="records"))
    flat_df = pd.json_normalize(json_struct)

    batch_id = f.name.split("_output.jsonl")[0]

    # extract response
    flat_df["model_response"] = flat_df["response.body.choices"].apply(
        lambda x: x[0]["message"]["content"]
    )
    flat_df["prompt_tokens"] = flat_df["response.body.usage.prompt_tokens"]
    flat_df["completion_tokens"] = flat_df["response.body.usage.completion_tokens"]
    flat_df["total_tokens"] = flat_df["response.body.usage.total_tokens"]
    flat_df["model"] = flat_df["response.body.model"]
    flat_df["batch_id"] = batch_id

    res_dfs.append(flat_df)
res_df = pd.concat(res_dfs, ignore_index=True)
batch_ids = res_df["batch_id"].unique()

for bid in batch_ids:
    batch = openai_client.batches.retrieve(bid)
    input_file = openai_client.files.retrieve(batch.input_file_id)
    res_df.loc[res_df["batch_id"] == bid, "input_file"] = input_file.filename

    res_df["grammar_name"] = res_df["input_file"].apply(
        lambda x: "_".join(str(x).split("_")[0:2])
    )

inputs_dfs = []

for f in inputs_files:
    df = pd.read_json(f, lines=True)
    json_struct = json.loads(df.to_json(orient="records"))
    flat_df = pd.json_normalize(json_struct)
    flat_df["grammar_name"] = flat_df["body.metadata.grammar_file"]
    flat_df["lhs"] = flat_df["body.metadata.lhs"]
    flat_df["rhs"] = flat_df["body.metadata.rhs"]
    inputs_dfs.append(flat_df)

inputs_df = pd.concat(inputs_dfs, ignore_index=True)


# join res_df and inputs_df on "grammar_name", "request_id"
res_df = pd.merge(
    res_df,
    inputs_df,
    on=["grammar_name", "custom_id"],
)

## Extract answer

In [ ]:
answer_re = re.compile(r"Final Answer: (.*?)(?:\n|$)", re.DOTALL)


def extract_answer(model_response):
    matches = answer_re.findall(model_response)
    if matches:
        last_match: str = matches[-1]
        last_match = re.sub(r"[^a-zA-Z\s]", "", last_match)
        last_match = last_match.strip()
        return last_match
    else:
        return None


# metrics
def exact_match(row) -> bool:
    return row["model_answer"] == row["rhs"]


def bow_match(row) -> bool:
    return sorted(row["model_answer"].split()) == sorted(row["rhs"].split())


def edit_distance(row) -> float:
    from strsimpy.jaro_winkler import JaroWinkler

    jw = JaroWinkler()
    return jw.distance(row["model_answer"], row["rhs"])

In [ ]:
res_df = res_df.drop_duplicates(subset=["custom_id", "batch_id"])
res_df["model_answer"] = res_df["model_response"].apply(extract_answer)
res_df = res_df.dropna(subset=["model_answer", "rhs"]).reset_index(drop=True)
res_df["exact_match"] = res_df["model_answer"] == res_df["rhs"]
res_df["bow_match"] = res_df.apply(
    lambda row: sorted(row["rhs"].split()) == sorted(row["model_answer"].split()),
    axis=1,
)
res_df["edit_distance"] = res_df.apply(edit_distance, axis=1)

res_df["lhs_length"] = res_df["lhs"].apply(lambda x: len(x.split()))


# melt exact_match and bow_match
metrics_df = res_df.melt(
    id_vars=["model", "lhs_length", "custom_id", "model_answer", "rhs"],
    value_vars=["exact_match", "bow_match", "edit_distance"],
    var_name="match_type",
    value_name="match_value",
)

# rename `exact_match` and `bow_match` in match_type column
metrics_df["match_type"] = metrics_df["match_type"].replace(
    {
        "exact_match": "Exact Match",
        "bow_match": "Bag of Words",
        "edit_distance": "Edit Distance",
    }
)

# convert the `model` column to an ordered categorical type
metrics_df["model_name"] = metrics_df["model"].apply(lambda x: x.split("-2")[0])
metrics_df["model_name"] = pd.Categorical(
    metrics_df["model_name"],
    categories=["gpt-4.1-nano", "gpt-4.1-mini", "o4-mini"],
    ordered=True,
)

In [ ]:
metrics_df[(metrics_df["model_name"] == "o4-mini") & (metrics_df["lhs_length"] == 20)]

In [ ]:
test_row = res_df[
    (res_df["edit_distance"] > 0.2) & (res_df["model"] != "o4-mini-2025-04-16")
]

test_row = test_row.iloc[5][
    ["model_response", "model_answer", "rhs", "edit_distance", "lhs_length"]
]

print(test_row["model_response"])

In [ ]:
fig = plt.figure(figsize=(7, 2.5), layout="constrained")
gs = fig.add_gridspec(1, 3)

ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[0, 1])
ax2 = fig.add_subplot(gs[0, 2])

axes = [ax0, ax1, ax2]


for i, model in enumerate(sorted(metrics_df["model_name"].unique())):
    model_df = metrics_df[metrics_df["model_name"] == model]
    sns.lineplot(
        data=model_df,
        x="lhs_length",
        y="match_value",
        hue="match_type",
        ax=axes[i],
        marker="o",
    )

    axes[i].set_ylim(-0.05, 1.05)

    model_name = str(model)

    # format y-axis ticks as percentages
    axes[i].set_yticks([0, 0.25, 0.5, 0.75, 1.0])
    axes[i].set_xticks([10, 20, 30, 40, 50])
    axes[i].set_title(model_name, fontsize=10)
    axes[i].set_ylabel("Score")
    axes[i].set_xlabel("Sentence Length", ha="left", x=0)
    # axes[i].set_xscale("log")

    if i > 0:
        axes[i].get_legend().remove()
        axes[i].set_ylabel(None)
        axes[i].set_yticks([])
        axes[i].set_xlabel(None)
    else:
        axes[i].legend(title=None, loc="upper left", fontsize=9)

In [ ]:
# histogram of rows by lhs_length
fig = plt.figure(figsize=(6, 2.5), layout="constrained")
gs = fig.add_gridspec(1, 3)

ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[0, 1])
ax2 = fig.add_subplot(gs[0, 2])

axes = [ax0, ax1, ax2]

for i, model in enumerate(sorted(metrics_df["model_name"].unique())):
    print(model)
    sns.histplot(
        data=metrics_df[metrics_df["model_name"] == model],
        x="lhs_length",
        bins=20,
        ax=axes[i],
        # kde=True,
        # stat="density",
        color="gray",
    )

    axes[i].set_yscale("log")
    axes[i].set_title(model, fontsize=10)
    # axes[i].set_ylabel("Accuracy")
    axes[i].set_xlabel("Sentence Length", ha="left", x=0)

    # horizontal line at y=1
    axes[i].axhline(y=1, color="red", linestyle="--", linewidth=0.5)

    if i > 0:
        # axes[i].get_legend().remove()
        axes[i].set_ylabel(None)
        axes[i].set_yticks([])
        axes[i].set_xlabel(None)
    else:
        pass
        # axes[i].legend(title=None, loc="upper left", fontsize=9)

In [ ]:
sns.lineplot(
    data=res_df,
    x="lhs_length",
    y="bow_match",
    hue="model",
    marker="o",
    ci=None,
    palette="tab10",
)